**Generator tesktów piosenek**



Import bibliotek

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk import flatten
import string, os, nltk, re, keras, io,sys, time,pickle,warnings
from tensorflow.keras import layers
from keras.layers import LSTM, Bidirectional, Dense, Dropout, Activation, Embedding, Flatten
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.optimizers import Adamax
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import datetime
import csv
from keras.utils import np_utils
warnings.filterwarnings("ignore")

In [2]:
start_time=time.time()

In [3]:
genres=['pop', 'rock', 'jazz', 'rap', 'country', 'blues']

Ustawienie parametrów do modelowania:

In [4]:
seq_len = 5
batch_size =  80
buffer_size = 1000
epochs = 40
embedding_dim = 250
rnn_units = 1050

Budowa modelu:

In [5]:
def split_text(x):
  input = x[:-1]
  target = x[1:]
  return input, target

for j in range(len(genres)):

  start_model_time=time.time()

  print(genres[j], ':')

  Words=[]
  df=pd.read_excel(f'dataset/df_{genres[j]}.xlsx')
  for l in df['lyrics']:
    Words.append(re.split(r' ', l))
  Words=flatten(Words)
  Words=[x for x in Words if x]

  dictionary = sorted(set(Words))
  dictionary_size = len(dictionary)
  dictionary_enumerated = {u: i for i, u in enumerate(dictionary)}
  enumerated_dictionary = np.array(dictionary)
  word_as_int = np.array([dictionary_enumerated[c] for c in Words])

  sequences = tf.data.Dataset.from_tensor_slices(word_as_int).batch(seq_len + 1, drop_remainder=True)
  dataset = sequences.map(split_text).shuffle(buffer_size).batch(batch_size, drop_remainder=True)


  def get_model(dictionary_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
      tf.keras.layers.Embedding(dictionary_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
      tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(dictionary_size)
  ])
    return model
  model = get_model(dictionary_size = len(dictionary), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=batch_size)

  def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

  model.compile(optimizer='adam', loss=loss)


  checkpoints = './checkpoints'
  callbacks=tf.keras.callbacks.ModelCheckpoint(
    filepath='./checkpoints/{epoch}',
    save_weights_only=True)

  history = model.fit(dataset, epochs=epochs, callbacks=[callbacks])

  tf.train.latest_checkpoint(checkpoints)
  model = get_model(len(dictionary), embedding_dim, rnn_units, batch_size=1)
  model.load_weights(tf.train.latest_checkpoint(checkpoints))
  model.build(tf.TensorShape([1, None]))
  model.summary()

  end_model_time=time.time()
  globals()[f'model_{genres[j]}_time']=end_model_time - start_model_time

  #zapisanie modelu
  pickle.dump(model, open(f'model_{genres[j]}.pkl','wb'))
  #files.download(f'model_{genres[j]}.pkl')

pop :
Epoch 1/40
1956/1956 [==============================] - 79s 36ms/step - loss: 5.0876
Epoch 2/40
1956/1956 [==============================] - 58s 30ms/step - loss: 4.4010
Epoch 3/40
1956/1956 [==============================] - 60s 31ms/step - loss: 4.0127
Epoch 4/40
1956/1956 [==============================] - 57s 29ms/step - loss: 3.7333
Epoch 5/40
1956/1956 [==============================] - 58s 30ms/step - loss: 3.5351
Epoch 6/40
1956/1956 [==============================] - 57s 29ms/step - loss: 3.3911
Epoch 7/40
1956/1956 [==============================] - 57s 29ms/step - loss: 3.2900
Epoch 8/40
1956/1956 [==============================] - 57s 29ms/step - loss: 3.2092
Epoch 9/40
1956/1956 [==============================] - 62s 31ms/step - loss: 3.1476
Epoch 10/40
1956/1956 [==============================] - 59s 30ms/step - loss: 3.0991
Epoch 11/40
1956/1956 [==============================] - 57s 29ms/step - loss: 3.0562
Epoch 12/40
1956/1956 [==============================] - 

Epoch 1/40
1219/1219 [==============================] - 43s 33ms/step - loss: 5.0752
Epoch 2/40
1219/1219 [==============================] - 28s 23ms/step - loss: 4.3680
Epoch 3/40
1219/1219 [==============================] - 29s 23ms/step - loss: 3.9734
Epoch 4/40
1219/1219 [==============================] - 27s 22ms/step - loss: 3.6736
Epoch 5/40
1219/1219 [==============================] - 27s 22ms/step - loss: 3.4498
Epoch 6/40
1219/1219 [==============================] - 27s 22ms/step - loss: 3.2819
Epoch 7/40
1219/1219 [==============================] - 28s 23ms/step - loss: 3.1572
Epoch 8/40
1219/1219 [==============================] - 27s 22ms/step - loss: 3.0568
Epoch 9/40
1219/1219 [==============================] - 28s 23ms/step - loss: 2.9810
Epoch 10/40
1219/1219 [==============================] - 27s 22ms/step - loss: 2.9187
Epoch 11/40
1219/1219 [==============================] - 27s 22ms/step - loss: 2.8677
Epoch 12/40
1219/1219 [==============================] - 27s 22

38/38 [==============================] - 10s 180ms/step - loss: 6.9427
Epoch 2/40
38/38 [==============================] - 4s 99ms/step - loss: 5.5792
Epoch 3/40
38/38 [==============================] - 3s 57ms/step - loss: 5.1591
Epoch 4/40
38/38 [==============================] - 2s 51ms/step - loss: 4.7114
Epoch 5/40
38/38 [==============================] - 1s 30ms/step - loss: 4.3129
Epoch 6/40
38/38 [==============================] - 1s 36ms/step - loss: 3.9543
Epoch 7/40
38/38 [==============================] - 1s 23ms/step - loss: 3.6402
Epoch 8/40
38/38 [==============================] - 1s 23ms/step - loss: 3.3618
Epoch 9/40
38/38 [==============================] - 1s 30ms/step - loss: 3.0824
Epoch 10/40
38/38 [==============================] - 1s 22ms/step - loss: 2.8291
Epoch 11/40
38/38 [==============================] - 1s 32ms/step - loss: 2.6099
Epoch 12/40
38/38 [==============================] - 1s 21ms/step - loss: 2.3839
Epoch 13/40
38/38 [===========================

805/805 [==============================] - 39s 45ms/step - loss: 5.6751
Epoch 2/40
805/805 [==============================] - 22s 28ms/step - loss: 4.8370
Epoch 3/40
805/805 [==============================] - 21s 26ms/step - loss: 4.3874
Epoch 4/40
805/805 [==============================] - 22s 27ms/step - loss: 4.0062
Epoch 5/40
805/805 [==============================] - 21s 27ms/step - loss: 3.6873
Epoch 6/40
805/805 [==============================] - 21s 26ms/step - loss: 3.4324
Epoch 7/40
805/805 [==============================] - 22s 27ms/step - loss: 3.2325
Epoch 8/40
805/805 [==============================] - 21s 26ms/step - loss: 3.0765
Epoch 9/40
805/805 [==============================] - 21s 26ms/step - loss: 2.9541
Epoch 10/40
805/805 [==============================] - 22s 27ms/step - loss: 2.8565
Epoch 11/40
805/805 [==============================] - 22s 27ms/step - loss: 2.7724
Epoch 12/40
805/805 [==============================] - 21s 26ms/step - loss: 2.7006
Epoch 13/40


479/479 [==============================] - 21s 39ms/step - loss: 5.4347
Epoch 2/40
479/479 [==============================] - 9s 18ms/step - loss: 4.6190
Epoch 3/40
479/479 [==============================] - 9s 19ms/step - loss: 4.2124
Epoch 4/40
479/479 [==============================] - 9s 18ms/step - loss: 3.8556
Epoch 5/40
479/479 [==============================] - 8s 16ms/step - loss: 3.5406
Epoch 6/40
479/479 [==============================] - 9s 18ms/step - loss: 3.2667
Epoch 7/40
479/479 [==============================] - 9s 18ms/step - loss: 3.0483
Epoch 8/40
479/479 [==============================] - 8s 17ms/step - loss: 2.8689
Epoch 9/40
479/479 [==============================] - 8s 17ms/step - loss: 2.7174
Epoch 10/40
479/479 [==============================] - 8s 17ms/step - loss: 2.6023
Epoch 11/40
479/479 [==============================] - 8s 17ms/step - loss: 2.5122
Epoch 12/40
479/479 [==============================] - 8s 16ms/step - loss: 2.4362
Epoch 13/40
479/479 [==

102/102 [==============================] - 11s 85ms/step - loss: 6.1229
Epoch 2/40
102/102 [==============================] - 4s 40ms/step - loss: 5.0489
Epoch 3/40
102/102 [==============================] - 2s 19ms/step - loss: 4.5758
Epoch 4/40
102/102 [==============================] - 2s 16ms/step - loss: 4.2113
Epoch 5/40
102/102 [==============================] - 2s 20ms/step - loss: 3.8862
Epoch 6/40
102/102 [==============================] - 2s 16ms/step - loss: 3.5864
Epoch 7/40
102/102 [==============================] - 2s 21ms/step - loss: 3.3006
Epoch 8/40
102/102 [==============================] - 1s 14ms/step - loss: 3.0426
Epoch 9/40
102/102 [==============================] - 2s 16ms/step - loss: 2.7984
Epoch 10/40
102/102 [==============================] - 2s 15ms/step - loss: 2.5865
Epoch 11/40
102/102 [==============================] - 2s 16ms/step - loss: 2.3936
Epoch 12/40
102/102 [==============================] - 2s 14ms/step - loss: 2.2284
Epoch 13/40
102/102 [==

Obliczenie ile czasu zajeło wykonanie skryptu oraz poszczególnych modeli:

In [6]:
for i in range(len(genres)):
  print("Czas wykonania modelu ", genres[i], ' ', datetime.timedelta(seconds=globals()[f'model_{genres[i]}_time']))

end_time=time.time()
total_time = end_time - start_time
print("Czas wykonania skryptu: ", datetime.timedelta(seconds=total_time))

Czas wykonania modelu  pop   0:44:59.751638
Czas wykonania modelu  rock   0:24:01.498662
Czas wykonania modelu  jazz   0:01:00.452478
Czas wykonania modelu  rap   0:17:40.430045
Czas wykonania modelu  country   0:05:58.857351
Czas wykonania modelu  blues   0:01:30.451353
Czas wykonania skryptu:  1:35:16.429376
